In [2]:
import pylab as plt
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table, join

In [3]:
import seaborn as sns

This notebook uses all the raw data from the CIGALE predictions and photoz catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in MOCs
The selection functions required are the main MOC associated with the masterlist. 

In [4]:
Sel_func=pymoc.MOC()
Sel_func.read('../../dmu4/dmu4_sm_SA13/data/holes_SA13_irac1_O16_20190201_WARNING-MADE-WITH-Lockman-SWIRE-PARAMS.fits')

## Read in CIGALE predictions catalogue

In [5]:
cigale=Table.read('../../dmu28/dmu28_SA13/data/SA13_Ldust_prediction_results.fits')

In [6]:
cigale['id'].name = 'help_id'

In [7]:
cigale

help_id,bayes.stellar.m_star,bayes.stellar.m_star_err,bayes.sfh.sfr10Myrs,bayes.sfh.sfr10Myrs_err,bayes.dust.luminosity,bayes.dust.luminosity_err,best.chi_square,best.reduced_chi_square,best.attenuation.Av_BC,best.attenuation.BC_to_ISM_factor,best.attenuation.bessell_b,best.attenuation.galex_fuv,best.attenuation.slope_BC,best.attenuation.slope_ISM,best.attenuation.stellar.old,best.attenuation.stellar.young,best.dust.alpha,best.dust.gamma,best.dust.luminosity,best.dust.mass,best.dust.qpah,best.dust.umin,best.lyc_absorption_f_dust,best.lyc_absorption_f_esc,best.sfh.age,best.sfh.burst_age,best.sfh.f_burst,best.sfh.integrated,best.sfh.sfr,best.sfh.sfr100Myrs,best.sfh.sfr10Myrs,best.sfh.tau_burst,best.sfh.tau_main,best.stellar.age_m_star,best.stellar.imf,best.stellar.lum_ly,best.stellar.lum_ly_old,best.stellar.lum_ly_young,best.stellar.m_gas,best.stellar.m_gas_old,best.stellar.m_gas_young,best.stellar.m_star,best.stellar.m_star_old,best.stellar.m_star_young,best.stellar.metallicity,best.stellar.n_ly,best.stellar.n_ly_old,best.stellar.n_ly_young,best.stellar.old_young_separation_age,best.universe.age,best.universe.luminosity_distance,best.universe.redshift,best.90prime_g,best.90prime_r,best.mosaic_z,best.ukidss_j
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,mJy,mJy,mJy,mJy
bytes27,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
HELP_J131038.491+424429.719,5566358258.231621,1890975895.91858,16.12772912167514,30.950496961946005,4.1303676205123843e+37,6.189354890175723e+37,0.9353263353917235,0.3117754451305745,1.7,0.3,0.7614924387291366,2.1764629981192005,-0.7,-0.7,3.280784283610898e+36,4.2963641517443844e+36,2.0,0.02,7.577148435355282e+36,4.1371029926130783e+37,0.47,5.0,0.0,0.0,4500.0,110.0,0.001,8678342470.984703,2.26642699646474,2.2787086249718427,2.2675546530602286,10000.0,3000.0,1822.4570713518153,1.0,9.259847439220629e+35,8.862191046354831e+33,9.17122552875708e+35,3544136759.0555596,3542663347.349813,1473411.7057462933,5134205696.645634,5113003577.104286,21202119.541347366,0.02,2.8322375912976046e+53,2.3753343736655096e+51,2.8084842475609496e+53,10.0,10618.614010620557,4.244436524969766e+25,0.27,0.01006591234065912,0.02060936485941919,0.030415387986579123,0.04184652079884462
HELP_J131040.639+424529.290,180482464550.87943,39752689514.26594,274.5097695134301,595.9779425903681,7.633510445591934e+38,1.1833155127607874e+39,9.305043624930008,3.1016812083100027,2.8,0.5,1.921376223543344,4.949035825600216,-0.7,-0.7,9.125156226363183e+37,1.1197444873961854e+38,2.0,0.02,2.032260110032504e+38,1.1096086416564217e+39,0.47,5.0,0.0,0.0,8000.0,10.0,0.001,315460274207.82434,57.65697917172377,29.550329558928677,57.69567381124142,10000.0,3000.0,3862.5078086081307,1.0,2.3529170371283416e+37,1.9666626503824714e+35,2.3332504106245166e+37,140072857724.90924,140035365367.59552,37492357.3137224,175387416094.11136,174847952102.1163,539463991.9950563,0.02,7.19264423006768e+54,4.7602417975229327e+52,7.145041812092451e+54,10.0,10428.166058206809,4.608777378296193e+25,0.29,0.04041794608091383,0.12122788602999551,0.2520157171161792,0.4293618028056324
HELP_J131041.208+425230.397,12337706833.781147,3832428277.1810627,32.191146660380916,66.57956184438952,8.261196654587904e+37,1.3204122833485516e+38,1.3032247436409048,0.4344082478803016,1.7,0.3,0.7297265423418866,2.179082816934455,-0.7,-0.7,3.9591052703394254e+36,4.160486207617973e+36,2.0,0.02,8.119591477957398e+36,4.433275458286325e+37,0.47,5.0,0.0,0.0,8000.0,110.0,0.001,23886271079.94949,2.194131072469409,2.2156917468574067,2.1960834607307285,10000.0,3000.0,3863.6562408238874,1.0,9.031104084638768e+35,1.51325816

## Read in photoz

In [8]:
photoz=Table.read('../../dmu24/dmu24_SA13/data/SA13_DESI-DR7_Zou_et_al_2019_photo_z_withhelp_id.fits')

In [9]:
photoz

help_id,id,ra,dec,photo_z,photo_zerr
,,deg,deg,,
bytes27,bytes16,float64,float64,float64,float64
HELP_J131305.238+423525.784,6000554456002869,198.27182520765405,42.59049542732427,0.73410904,0.042880192
HELP_J131212.836+422748.695,6000554455001280,198.0534815684315,42.46352631919768,0.12239093,0.041696854
HELP_J131231.867+425850.944,6000556575001377,198.13277725855303,42.98081774414297,0.89414376,0.10019907
HELP_J131150.019+423817.385,6000555516000167,197.9584139674228,42.638162483787724,0.41027397,0.083354525
HELP_J131220.528+425817.884,6000556575001236,198.08553458782316,42.97163440538101,0.2124581,0.088154316
HELP_J131142.989+424528.110,6000555516001659,197.9291205263603,42.757808207283375,0.5794851,0.038273126
HELP_J131230.871+425539.321,6000556575000687,198.12862926819162,42.927589184709404,0.9508035,0.106143564
HELP_J131102.039+425500.816,6000556574000533,197.75849612671212,42.916893320260755,0.72463334,0.19623348


## Join CIGALE and photoz tables

In [10]:
prior=join(cigale,photoz)

In [11]:
from astropy.cosmology import Planck15 as cosmo
from astropy import units as u
f_pred=prior['bayes.dust.luminosity']/(4*np.pi*cosmo.luminosity_distance(prior['photo_z']).to(u.cm))

In [12]:
prior=prior[np.isfinite(f_pred.value)][np.log10(f_pred.value[np.isfinite(f_pred.value)])>8.5]

In [13]:
prior['dec'].name='Dec'
prior['ra'].name='RA'

In [14]:
len(cigale)

833

## Read in Maps

In [15]:
pswfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/SA13_SPIRE250_v1.0.fits'#SPIRE 250 map
pmwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/SA13_SPIRE350_v1.0.fits'#SPIRE 350 map
plwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/SA13_SPIRE500_v1.0.fits'#SPIRE 500 map

#output folder
output_folder='./'

In [16]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist['PRIMARY'].header
im250hdu=hdulist['IMAGE'].header

im250=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim250=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist['IMAGE'].header)
pixsize250=3600.0*w_250.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist['PRIMARY'].header
im350hdu=hdulist['IMAGE'].header

im350=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim350=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist['IMAGE'].header)
pixsize350=3600.0*w_350.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist['PRIMARY'].header
im500hdu=hdulist['IMAGE'].header

im500=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim500=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist['IMAGE'].header)
pixsize500=3600.0*w_500.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()

In [17]:
## Set XID+ prior class

In [18]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=Sel_func)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(prior['RA'] ,prior['Dec'] ,'SA13_Ldust_prediction.fits',ID=prior['help_id'] )#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=Sel_func)
prior350.prior_cat(prior['RA'] ,prior['Dec'] ,'SA13_Ldust_prediction.fits',ID=prior['help_id'] )
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=Sel_func)
prior500.prior_cat(prior['RA'] ,prior['Dec'] ,'SA13_Ldust_prediction',ID=prior['help_id'] )
prior500.prior_bkg(-5.0,5)

Private attributes "_naxis1" and "_naxis2" have been deprecated since v3.1.
Instead use the "pixel_shape" property which returns a list of NAXISj keyword values.
 [astropy.wcs.wcs]
Private attributes "_naxis1" and "_naxis2" have been deprecated since v3.1.
Instead use the "pixel_shape" property which returns a list of NAXISj keyword values.
 [astropy.wcs.wcs]


In [19]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)





##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [20]:
pixsize

array([ 6.00000015,  8.3333334 , 12.00000029])

In [21]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines
order=9
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
output_folder='./'
outfile=output_folder+'Master_prior.pkl'
xidplus.io.pickle_dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},outfile)
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)
raise SystemExit()

----- There are 27 tiles required for input catalogue and 2 large tiles
writing total_bytes=4637647...
writing bytes [0, 4637647)... done.


SystemExit: 

In [22]:
prior250.nsrc

812